# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	note
2020-10-25	Securing the Internet of Things from the Bottom Up Using Physical Unclonable Functions	The Eleventh International Conference on Cloud Computing, GRIDs, and Virtualization, Proceedings	This paper is about Physical Unclonable Functions and their use in Internet of Things devices.	L. Lathrop, S. Liebl, U. Raithel, M. Söllner, and A. Aßmuth, “Securing the Internet of Things from the Bottom Up Using Physical Unclonable Functions,” in The Eleventh International Conference on Cloud Computing, GRIDs, and Virtualization, Proceedings, Nice, France, 25th to 29th October 2020, p. 44 to 49.	CC2020-2	https://www.researchgate.net/publication/344900322_Securing_the_Internet_of_Things_from_the_Bottom_Up_Using_Physical_Unclonable_Functions	This work was mainly done by my great colleague Leah Lathrop, have a look at her [papers](https://www.researchgate.net/profile/Leah-Lathrop).
2020-10-26	Threat Analysis of Industrial Internet of 

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,note
0,2020-10-25,Securing the Internet of Things from the Botto...,The Eleventh International Conference on Cloud...,This paper is about Physical Unclonable Functi...,"L. Lathrop, S. Liebl, U. Raithel, M. Söllner, ...",CC2020-2,https://www.researchgate.net/publication/34490...,This work was mainly done by my great colleagu...
1,2020-10-26,Threat Analysis of Industrial Internet of Thin...,The Eleventh International Conference on Cloud...,This paper is about the procedure for the thre...,"S. Liebl, L. Lathrop, U. Raithel, M. Söllner, ...",CC2020-1,https://www.researchgate.net/publication/34506...,This paper is about the procedure for the thre...
2,2021-02-01,Securing Low-Power Industrial Devices in Criti...,Research Report 2021 of the Technical Universi...,This report summarizes our research project “i...,"S. Liebl, L. Lathrop, A. Aßmuth, and M. Söllne...",OTH2021-1,https://www.researchgate.net/publication/35110...,We described the progress of our 3-year resear...
3,2021-04-18,A Secure and Privacy-Friendly Logging Scheme,The Twelfth International Conference on Cloud ...,This paper is about a secure and privacy-frien...,"A. Aßmuth, R. Duncan, S. Liebl, and M. Söllner...",CC2021-1,https://www.researchgate.net/publication/35108...,I came up with this idea while I was thinking ...
4,2021-12-31,Analyzing the Attack Surface and Threats of In...,International Journal on Advances in Security,This paper is an extension of our paper about ...,"S. Liebl, L. Lathrop, U. Raithel, A. Aßmuth, I...",AiS2021-1,https://www.researchgate.net/publication/35828...,This paper describes the characteristics of II...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    #if len(str(item.excerpt)) > 5:
    #    md += "\n" + html_escape(item.excerpt) + "\n"
    
    #if len(str(item.paper_url)) > 5:
    #    md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    #md += "\nRecommended citation: " + item.citation + "\n"
    
    #md_filename = os.path.basename(md_filename)
    
    if len(str(item.note)) > 5:
        md += "\n" + html_escape(item.note) + "\n"
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2020-10-25-CC2020-2.md	2021-02-01-OTH2021-1.md  2021-12-30-AiS2021-1.md
2020-10-26-CC2020-1.md	2021-04-18-CC2021-1.md	 2021-12-31-AiS2021-1.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
